# Golf Predictor 

## (0) Load Data

In [1]:
import pandas as pd
# dfRaw is raw dataset read from csv
dfRaw = pd.read_csv("../data_2015_thru_2019.csv")
dfRaw

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,...,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK FOLLOWING YEAR,RANK AS LABEL
0,Jason Day,2015,75,0.287,16.357,57,75,1.520,0.772,0.461,...,2.106,120.054,86.649,33.407,0.586,33.407,0.461,26.260,1,A
1,Dustin Johnson,2015,75,-0.212,-11.466,54,75,1.326,0.960,0.579,...,1.455,78.552,71.623,6.932,0.128,6.932,0.579,31.252,2,A
2,Jordan Spieth,2015,91,0.471,31.054,66,91,1.583,0.494,0.618,...,2.154,142.131,104.470,37.665,0.571,37.665,0.618,40.776,4,A
3,Henrik Stenson,2015,63,0.082,3.604,44,63,1.774,0.448,1.244,...,2.210,97.245,78.041,19.205,0.436,19.205,1.244,54.732,5,A
4,Adam Scott,2015,51,-0.195,-6.238,32,51,0.601,0.682,0.114,...,0.206,6.577,19.240,-12.662,-0.396,-12.662,0.114,3.659,6,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,Patton Kizzire,2019,76,-0.120,-6.337,53,76,-0.740,-0.489,-0.132,...,-0.098,-5.192,-39.241,34.050,0.642,34.050,-0.132,-6.987,289,F
632,Peter Malnati,2019,87,0.097,7.362,76,87,-0.088,-0.410,0.225,...,0.379,28.774,-6.693,35.469,0.467,35.469,0.225,17.084,290,F
633,Bronson Burgoon,2019,59,-0.074,-3.481,47,59,-0.053,0.054,-0.033,...,-0.344,-16.148,-2.469,-13.679,-0.291,-13.679,-0.033,-1.540,293,F
634,Chris Stroud,2019,70,0.035,1.808,52,70,-0.752,-0.557,-0.229,...,-0.729,-37.920,-39.088,1.165,0.022,1.165,-0.229,-11.918,298,F


### (0.1) Load Training Data from File

In [2]:
# --------- For Now we will be using 2015-2018 as TRAINING, 2019 as TESTING --------------
dfTrain = pd.read_csv("../training.csv")
# delete rank next year column
dfTrain = dfTrain.drop(['RANK FOLLOWING YEAR'], axis=1)
dfTrain

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,SG:ARG,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK AS LABEL
0,Jason Day,2015,75,0.287,16.357,57,75,1.520,0.772,0.461,0.287,2.106,120.054,86.649,33.407,0.586,33.407,0.461,26.260,A
1,Dustin Johnson,2015,75,-0.212,-11.466,54,75,1.326,0.960,0.579,-0.212,1.455,78.552,71.623,6.932,0.128,6.932,0.579,31.252,A
2,Jordan Spieth,2015,91,0.471,31.054,66,91,1.583,0.494,0.618,0.471,2.154,142.131,104.470,37.665,0.571,37.665,0.618,40.776,A
3,Henrik Stenson,2015,63,0.082,3.604,44,63,1.774,0.448,1.244,0.082,2.210,97.245,78.041,19.205,0.436,19.205,1.244,54.732,A
4,Adam Scott,2015,51,-0.195,-6.238,32,51,0.601,0.682,0.114,-0.195,0.206,6.577,19.240,-12.662,-0.396,-12.662,0.114,3.659,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,Johnson Wagner,2018,75,0.226,12.456,55,75,0.092,-0.213,0.078,0.226,0.715,39.339,5.049,34.292,0.623,34.292,0.078,4.304,F
500,Scott Brown,2018,102,-0.175,-14.172,81,102,-0.710,-0.265,-0.270,-0.175,-0.661,-53.530,-57.498,3.967,0.049,3.967,-0.270,-21.856,F
501,Trey Mullinax,2018,75,-0.207,-12.199,59,75,-0.144,0.506,-0.442,-0.207,-0.072,-4.231,-8.472,4.243,0.072,4.243,-0.442,-26.106,F
502,Shawn Stefani,2018,76,0.013,0.809,60,76,0.141,-0.093,0.220,0.013,0.107,6.420,8.464,-2.045,-0.034,-2.045,0.220,13.227,F


### (0.2) Load Testing Data from File

In [3]:
dfTest = pd.read_csv("../testing.csv")
dfTest = dfTest.drop("RANK FOLLOWING YEAR", axis=1)
dfTest

,PLAYER NAME,YEAR,ROUNDS,AVERAGE,TOTAL SG:ARG,MEASURED ROUNDS,ROUNDS_x,AVERAGE_x,SG:OTT,SG:APR,SG:ARG,AVERAGE_y,TOTAL SG:T,TOTAL SG:T2G,TOTAL SG:P,AVERAGE_x.1,TOTAL SG:PUTTING,AVERAGE_y.1,TOTAL SG:APP,RANK AS LABEL
0,Dustin Johnson,2019,73,0.234,13.098,56,73,1.225,0.703,0.288,0.234,1.321,73.995,68.619,5.378,0.096,5.378,0.288,16.143,A
1,Jon Rahm,2019,75,0.073,3.863,53,75,1.170,0.692,0.405,0.073,1.505,79.788,61.994,17.793,0.336,17.793,0.405,21.449,A
2,Justin Thomas,2019,75,0.352,19.729,56,75,1.817,0.479,0.985,0.352,1.631,91.314,101.736,-10.419,-0.186,-10.419,0.985,55.176,A
3,Rory McIlroy,2019,72,0.297,16.939,57,72,2.126,1.195,0.633,0.297,2.551,145.379,121.159,24.221,0.425,24.221,0.633,36.096,A
4,Webb Simpson,2019,81,0.294,20.269,69,81,0.899,-0.007,0.613,0.294,1.507,104.007,62.051,41.957,0.608,41.957,0.613,42.285,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,Patton Kizzire,2019,76,-0.120,-6.337,53,76,-0.740,-0.489,-0.132,-0.120,-0.098,-5.192,-39.241,34.050,0.642,34.050,-0.132,-6.987,F
128,Peter Malnati,2019,87,0.097,7.362,76,87,-0.088,-0.410,0.225,0.097,0.379,28.774,-6.693,35.469,0.467,35.469,0.225,17.084,F
129,Bronson Burgoon,2019,59,-0.074,-3.481,47,59,-0.053,0.054,-0.033,-0.074,-0.344,-16.148,-2.469,-13.679,-0.291,-13.679,-0.033,-1.540,F
130,Chris Stroud,2019,70,0.035,1.808,52,70,-0.752,-0.557,-0.229,0.035,-0.729,-37.920,-39.088,1.165,0.022,1.165,-0.229,-11.918,F


### (0.3) Find Feature Names

In [4]:
featureNames = dfTrain.columns[3:-1] # Skip ID, name, year, rounds, and rank as label
nFeatures = featureNames.shape[0]
print("Number of features:", nFeatures)
featureNames

Number of features: 16


Index(['AVERAGE', 'TOTAL SG:ARG', 'MEASURED ROUNDS', 'ROUNDS_x', 'AVERAGE_x',
       'SG:OTT', 'SG:APR', 'SG:ARG', 'AVERAGE_y', 'TOTAL SG:T', 'TOTAL SG:T2G',
       'TOTAL SG:P', 'AVERAGE_x.1', 'TOTAL SG:PUTTING', 'AVERAGE_y.1',
       'TOTAL SG:APP'],
      dtype='object')

### (0.4) Select Training Instances (features and values) from the Training Data

In [5]:
trainingInstances_x = []
trainingInstances_y = []
for instance in dfTrain.to_numpy():
    featureValues = list(instance[3:-1])
    label = instance[-1]
    trainingInstances_x.append(featureValues)
    trainingInstances_y.append(label)
nTrainingInstances = len(trainingInstances_x)
print("Number of training instances:", nTrainingInstances)
trainingInstances_x
trainingInstances_y

Number of training instances: 504


['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C'

### (0.5) Select Test Instances from the Data

In [6]:
testInstances_x = []
testInstances_y = []
for instance in dfTrain.to_numpy():
    featureValues = list(instance[3:-1])
    label = instance[-1]
    testInstances_x.append(featureValues)
    testInstances_y.append(label)
nTrainingInstances = len(trainingInstances_x)
print("Number of training instances:", nTrainingInstances)
testInstances_x
testInstances_y

Number of training instances: 504


['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'E',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'F',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C'

## 1. Support Vector Classification

### 1.1 Linear SVM

In [7]:
from sklearn.svm import LinearSVC
from sklearn import metrics
clf = LinearSVC()
clf.fit(trainingInstances_x, trainingInstances_y)
y_pred = clf.predict(testInstances_x)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))

y_pred_svm = y_pred

Predicted: ['B' 'B' 'B' 'B' 'B' 'A' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'F' 'F' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'F' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'F' 'B' 'B' 'B' 'B' 'B' 'B'
 'E' 'B' 'B' 'F' 'F' 'F' 'B' 'B' 'B' 'B' 'B' 'F' 'B' 'B' 'B' 'F' 'B' 'F'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'A' 'B' 'B' 'A' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'B' 'F' 'B' 'B' 'F' 'B' 'B' 'B' 'B'
 'B' 'B' 'B' 'B' 'E' 'B' 'B' 'B' 'B' 'F' 'B' 'B' 'B' 'B' 'B' 'B' 'F' 'B'
 'B' 'F' 'B' 'B' 'F' 'F' 'B' 'B' 'B' 'B'

/Users/michaellee/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


### 1.2 Kernel SVM  

Kernel is Radius Basis Function: Gaussian:  

In [8]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(trainingInstances_x, trainingInstances_y)
y_pred = clf.predict(testInstances_x)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))

Predicted: ['A' 'A' 'A' 'A' 'F' 'A' 'B' 'B' 'A' 'A' 'A' 'E' 'B' 'A' 'A' 'B' 'E' 'A'
 'A' 'E' 'F' 'A' 'E' 'A' 'E' 'A' 'E' 'E' 'E' 'B' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'B' 'E' 'E' 'E' 'E' 'B' 'B' 'F' 'B' 'E' 'E' 'F' 'E' 'E' 'E' 'D' 'A'
 'E' 'E' 'B' 'B' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'A'
 'E' 'E' 'E' 'E' 'F' 'E' 'F' 'E' 'E' 'E' 'B' 'F' 'F' 'E' 'E' 'E' 'F' 'E'
 'E' 'F' 'F' 'E' 'E' 'A' 'E' 'E' 'E' 'E' 'E' 'F' 'E' 'F' 'E' 'E' 'E' 'E'
 'F' 'F' 'E' 'F' 'F' 'F' 'E' 'E' 'F' 'F' 'E' 'F' 'E' 'E' 'E' 'F' 'E' 'F'
 'E' 'E' 'E' 'F' 'A' 'A' 'A' 'E' 'A' 'A' 'A' 'D' 'A' 'A' 'A' 'A' 'E' 'E'
 'A' 'D' 'A' 'E' 'E' 'B' 'B' 'A' 'F' 'A' 'E' 'B' 'E' 'B' 'E' 'B' 'D' 'E'
 'E' 'B' 'B' 'E' 'B' 'E' 'B' 'E' 'A' 'E' 'E' 'E' 'E' 'B' 'E' 'E' 'E' 'A'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'B' 'E' 'F' 'E' 'E' 'F' 'F' 'B' 'E' 'F' 'E' 'E' 'E' 'E'
 'B' 'E' 'B' 'E' 'E' 'E' 'E' 'F' 'E' 'F' 'E' 'F' 'E' 'E' 'E' 'E' 'F' 'E'
 'E' 'F' 'F' 'E' 'F' 'F' 'F' 'E' 'E' 'E'

Kernel is Quadratic kernel ("Degree-2 polynomial kernel"):

In [9]:
clf = SVC(kernel='poly', degree=2)
clf.fit(trainingInstances_x, trainingInstances_y)
y_pred = clf.predict(testInstances_x)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))

y_pred_kernel = y_pred

Predicted: ['A' 'A' 'A' 'A' 'E' 'A' 'B' 'B' 'A' 'D' 'B' 'E' 'B' 'A' 'A' 'B' 'E' 'B'
 'A' 'E' 'E' 'A' 'E' 'B' 'E' 'B' 'E' 'B' 'E' 'B' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'B' 'E' 'E' 'E' 'E' 'B' 'B' 'E' 'B' 'E' 'E' 'E' 'E' 'E' 'E' 'D' 'A'
 'E' 'E' 'B' 'B' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'A'
 'E' 'E' 'E' 'E' 'F' 'E' 'E' 'E' 'E' 'E' 'B' 'F' 'F' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'F' 'E' 'E' 'A' 'E' 'E' 'E' 'E' 'E' 'F' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'F' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'F' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'A' 'A' 'A' 'E' 'A' 'A' 'A' 'E' 'A' 'A' 'A' 'A' 'E' 'E'
 'B' 'E' 'A' 'E' 'E' 'B' 'B' 'B' 'E' 'A' 'E' 'B' 'E' 'B' 'E' 'B' 'E' 'E'
 'E' 'B' 'B' 'E' 'B' 'E' 'B' 'E' 'A' 'E' 'E' 'E' 'E' 'B' 'E' 'E' 'E' 'B'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'D' 'E' 'E' 'E' 'E' 'E' 'F' 'B' 'E' 'E' 'E' 'E' 'E' 'E'
 'B' 'E' 'B' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'F' 'E' 'E' 'E' 'E'

## 2. KNN

### Training KNN Classifiers

In [10]:
# KNN WITH n_neighbors=5
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

clf = KNeighborsClassifier() # n_neighbors=5
clf.fit(trainingInstances_x, trainingInstances_y)
y_pred = clf.predict(testInstances_x)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))

Predicted: ['A' 'A' 'A' 'A' 'D' 'A' 'C' 'A' 'A' 'A' 'A' 'E' 'B' 'A' 'A' 'A' 'A' 'B'
 'A' 'B' 'D' 'A' 'E' 'A' 'B' 'A' 'D' 'D' 'D' 'E' 'D' 'E' 'C' 'F' 'D' 'D'
 'F' 'E' 'D' 'C' 'E' 'D' 'A' 'D' 'F' 'D' 'D' 'E' 'E' 'D' 'B' 'E' 'C' 'A'
 'B' 'E' 'A' 'A' 'D' 'E' 'B' 'F' 'A' 'E' 'E' 'E' 'E' 'D' 'F' 'E' 'E' 'B'
 'E' 'E' 'E' 'E' 'F' 'E' 'E' 'E' 'F' 'D' 'E' 'E' 'F' 'E' 'B' 'E' 'E' 'E'
 'E' 'F' 'F' 'E' 'B' 'D' 'D' 'E' 'E' 'E' 'F' 'E' 'F' 'F' 'F' 'F' 'E' 'E'
 'E' 'D' 'F' 'F' 'F' 'F' 'F' 'E' 'F' 'F' 'B' 'F' 'F' 'F' 'F' 'F' 'F' 'F'
 'C' 'D' 'F' 'F' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'E' 'A'
 'A' 'A' 'B' 'B' 'C' 'A' 'C' 'A' 'E' 'B' 'B' 'A' 'A' 'E' 'B' 'A' 'A' 'B'
 'D' 'C' 'C' 'C' 'C' 'E' 'B' 'E' 'A' 'E' 'C' 'C' 'D' 'D' 'E' 'C' 'E' 'D'
 'E' 'D' 'E' 'D' 'D' 'D' 'E' 'D' 'C' 'F' 'C' 'E' 'E' 'D' 'E' 'E' 'F' 'D'
 'E' 'E' 'E' 'E' 'B' 'E' 'E' 'E' 'C' 'E' 'E' 'D' 'D' 'F' 'A' 'E' 'E' 'E'
 'B' 'F' 'B' 'E' 'E' 'C' 'E' 'E' 'F' 'F' 'E' 'F' 'F' 'F' 'F' 'C' 'F' 'E'
 'F' 'F' 'E' 'A' 'F' 'E' 'F' 'E' 'E' 'E'

In [11]:
# KNN WITH n_neighbors=3
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

clf = KNeighborsClassifier(n_neighbors=3) # n_neighbors = 3
clf.fit(trainingInstances_x, trainingInstances_y)
y_pred = clf.predict(testInstances_x)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(testInstances_y, y_pred))

Predicted: ['A' 'A' 'A' 'A' 'A' 'A' 'C' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'C' 'B'
 'A' 'B' 'B' 'A' 'A' 'A' 'B' 'B' 'B' 'B' 'B' 'E' 'D' 'E' 'A' 'C' 'C' 'B'
 'C' 'E' 'D' 'C' 'C' 'C' 'A' 'D' 'F' 'A' 'D' 'E' 'D' 'D' 'C' 'D' 'C' 'A'
 'B' 'C' 'A' 'B' 'D' 'E' 'B' 'B' 'A' 'D' 'A' 'E' 'E' 'A' 'C' 'E' 'E' 'B'
 'F' 'E' 'E' 'E' 'F' 'D' 'D' 'E' 'F' 'D' 'E' 'E' 'F' 'E' 'E' 'C' 'D' 'E'
 'E' 'F' 'F' 'E' 'E' 'C' 'D' 'E' 'E' 'E' 'F' 'E' 'A' 'F' 'F' 'E' 'F' 'F'
 'F' 'D' 'F' 'F' 'F' 'F' 'F' 'D' 'F' 'F' 'B' 'F' 'F' 'F' 'F' 'F' 'F' 'F'
 'C' 'D' 'F' 'F' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'A' 'B' 'B' 'A' 'A' 'A' 'A'
 'A' 'A' 'B' 'B' 'B' 'B' 'C' 'A' 'F' 'B' 'B' 'A' 'B' 'B' 'B' 'C' 'C' 'C'
 'B' 'A' 'C' 'C' 'C' 'C' 'A' 'E' 'A' 'C' 'C' 'B' 'D' 'B' 'D' 'D' 'E' 'D'
 'E' 'E' 'C' 'D' 'D' 'D' 'E' 'D' 'C' 'D' 'C' 'D' 'E' 'B' 'E' 'D' 'F' 'D'
 'E' 'E' 'E' 'C' 'E' 'E' 'F' 'D' 'E' 'E' 'E' 'B' 'A' 'F' 'A' 'F' 'E' 'E'
 'B' 'F' 'B' 'E' 'E' 'C' 'E' 'D' 'F' 'F' 'F' 'F' 'F' 'D' 'C' 'F' 'F' 'E'
 'F' 'F' 'C' 'A' 'F' 'E' 'F' 'F' 'E' 'C'

## X. Categorical Naive Bayes

In [12]:
from sklearn.naive_bayes import CategoricalNB
# NOTE---- FAILS DUE TO NEGATIVE DATA IN X
clf = CategoricalNB()
clf.fit(trainingInstances_x, trainingInstances_y)
y_pred = clf.predict(testInstances_x)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(testInstance_y, y_pred))

y_pred_bayes = y_pred

ValueError: Negative values in data passed to CategoricalNB (input X)